### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [97]:
# Dependencies and Setup
import numpy as np
import pandas as pd
import os

# Load CSV
csv_path = os.path.join("Resources", "purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(csv_path)

#View the content
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [98]:
total_players = purchase_data["SN"].nunique()
total_players_df = pd.DataFrame({
    "Total Players" : [total_players]
    })
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [99]:
#Basic calculations
unique_items = purchase_data["Item ID"].nunique()
average_price = purchase_data["Price"].mean()
nr_purchases = purchase_data["Purchase ID"].count()
total_revenue = purchase_data["Price"].sum()

In [100]:
#Summary Data Frame

calculations_df = pd.DataFrame(
   {"Number of Unique Items" : [unique_items],
    "Average Price" : [average_price],
    "Number of Purchases" : [nr_purchases],
    "Total Revenue" : [total_revenue]
   })

In [101]:
#Use Map to format Average Price & Total Revenue

calculations_df["Average Price"] = calculations_df["Average Price"].map("${:.2f}".format)
calculations_df["Total Revenue"] = calculations_df["Total Revenue"].map("${:,.2f}".format)

#Print Calculations

calculations_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [102]:
#Calculate Gender Demographics

gender = purchase_data.groupby("Gender")

gender_distinction = gender["SN"].nunique()

gender_percentage = gender["SN"].nunique()/purchase_data["SN"].nunique()

In [103]:
#Merge both data to one df

gender_data = pd.merge(gender_distinction, gender_percentage, on="Gender")

In [104]:
#Rename columns to understand which data is displayed

gender_data = gender_data.rename(columns={"SN_x":"Total Count", "SN_y":"Percentage of Players"})

#Sort ASC by Total Count

gender_data = gender_data\
    .sort_values("Total Count", ascending = False)

In [105]:
#Use Map to format Percentage of Players

gender_data["Percentage of Players"] = gender_data["Percentage of Players"].map("{:.2%}".format)

#Print Gender Demographics Data
gender_data

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [106]:
#Basic Purchasing Calculations
purchase_count = gender["Purchase ID"].count()

average_price = gender["Price"].mean()

total_purchase = gender["Price"].sum()

In [107]:
#Merge data 

purchasing_data = pd.merge(purchase_count, average_price, on="Gender")

purchasing_data = pd.merge(purchasing_data, total_purchase, on="Gender")

In [108]:
#Rename columns for a better understanding

purchasing_data = purchasing_data.rename(columns = {"Purchase ID" : "Number of Purchases",
                                                    "Price_x" : "Average Purchase Price",
                                                    "Price_y" : "Total Purchase"
                                                   })

In [109]:
#Calculate Purchase per person

purchase_per_person = purchasing_data["Total Purchase"]/gender["SN"].nunique()

#Add calculation purchase per person in a column to the Data Frame

purchasing_data["Purchase per Person"] = purchase_per_person

In [110]:
#Use Map to format AVG Purchase Price, Total Purchase & Purchase per Person

purchasing_data["Average Purchase Price"] = purchasing_data["Average Purchase Price"].map("${:.2f}".format)

purchasing_data["Total Purchase"] = purchasing_data["Total Purchase"].map("${:,.2f}".format)

purchasing_data["Purchase per Person"] = purchasing_data["Purchase per Person"].map("${:.2f}".format)

In [111]:
#Sort by higher total purchase

purchasing_data = purchasing_data\
    .sort_values("Total Purchase")

#Print Purchase df

purchasing_data

,Number of Purchases,Average Purchase Price,Total Purchase,Purchase per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [112]:
# Display a statistical overview of age in the Data Frame to understand better it distribution

purchase_data["Age"].describe()

count    780.000000
mean      22.714103
std        6.659444
min        7.000000
25%       20.000000
50%       22.000000
75%       25.000000
max       45.000000
Name: Age, dtype: float64

In [113]:
#Assign minimum and maximum variables for Ages

min_age = purchase_data["Age"].min()

max_age = purchase_data["Age"].max()       

In [114]:
#Create bins in which to place age values

bins = np.linspace(min_age, max_age, num = 6, endpoint = True)

#Bin Labels

bin_labels = ["<15", "15-22", "23-29", "30-37", "39+"]

In [115]:
#Slice the data and place it into bins

purchase_data["Bin"] = pd.cut(purchase_data["Age"], bins, labels = bin_labels, include_lowest=True)


In [116]:
#Group data frame by bins
bin_df = purchase_data.groupby("Bin")

#Find how many players fall into each bin

bin_df = bin_df["SN"].nunique()

#Calculate the percentage of total for each bin

percentages_bin = bin_df/purchase_data["SN"].nunique()

#Merge values

bin_df = pd.merge(bin_df, percentages_bin, on="Bin")

In [117]:
#Rename columns name

bin_df = bin_df.rename(columns={
    "SN_x" : "Nr of Players",
    "SN_y" : "Percentage of Total"
})


In [118]:
#Format percentages with map

bin_df["Percentage of Total"] = bin_df["Percentage of Total"].map("{:.2%}".format)

#Print bin data frame
bin_df


,Nr of Players,Percentage of Total
Bin,,
<15,39,6.77%
15-22,268,46.53%
23-29,174,30.21%
30-37,72,12.50%
39+,23,3.99%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [119]:
# Bin's Purchase data in "purchase_data" data frame

#Group data frame by bins
bin_df = purchase_data.groupby("Bin")

In [142]:

purchase_count = bin_df["Price"].count()

average_purchase = bin_df["Price"].mean()

total_purchase =  bin_df["Price"].sum()

players_purchase = bin_df["SN"].nunique()


In [143]:
#Merge data 

purchasing_data_age = pd.merge(purchase_count, average_purchase, on="Bin")

purchasing_data_age = pd.merge(purchasing_data_age, total_purchase, on="Bin")

purchasing_data_age

,Price_x,Price_y,Price
Bin,,,
<15,51,3.135490,159.91
15-22,367,3.037602,1114.80
23-29,235,3.000638,705.15
30-37,99,3.072727,304.20
39+,28,3.418214,95.71


In [144]:
#Rename columns for a better understanding

purchasing_data_age = purchasing_data_age.rename(columns = {"Price_x" : "Number of Purchases",
                                                    "Price_y" : "Average Purchase Price",
                                                    "Price" : "Total Purchase"
                                                   })

purchasing_data_age

,Number of Purchases,Average Purchase Price,Total Purchase
Bin,,,
<15,51,3.135490,159.91
15-22,367,3.037602,1114.80
23-29,235,3.000638,705.15
30-37,99,3.072727,304.20
39+,28,3.418214,95.71


In [145]:
#Calculate Purchase per person

purchasing_per_person = purchasing_data_age["Total Purchase"]/players_purchase

In [146]:
#Add calculation purchase per person in a column to the Data Frame

purchasing_data_age["Purchase per Person"] = purchasing_per_person

In [147]:
#Use Map to format AVG Purchase Price, Total Purchase & Purchase per Person

purchasing_data_age["Average Purchase Price"] = purchasing_data_age["Average Purchase Price"].map("${:.2f}".format)

purchasing_data_age["Total Purchase"] = purchasing_data_age["Total Purchase"].map("${:,.2f}".format)

purchasing_data_age["Purchase per Person"] = purchasing_data_age["Purchase per Person"].map("${:.2f}".format)


#Print Summary Data Frame

purchasing_data_age


,Number of Purchases,Average Purchase Price,Total Purchase,Purchase per Person
Bin,,,,
<15,51,$3.14,$159.91,$4.10
15-22,367,$3.04,"$1,114.80",$4.16
23-29,235,$3.00,$705.15,$4.05
30-37,99,$3.07,$304.20,$4.22
39+,28,$3.42,$95.71,$4.16


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
